### Imports

In [13]:
# NYTimes News Data
from bs4 import BeautifulSoup
import requests

# Utility
from pathlib import Path
from os import path
import json
import time
import random

### Functions

In [17]:
def get_filepaths(directory_path: str):
    # Get all .json files (including subdirectories) that do not start with 'fulltext'
    json_filepaths = [str(file) for file in Path(directory_path).rglob("*.json") if not file.name.startswith("fulltext")]

    return json_filepaths

In [21]:
def filter_scraped_files(directory_path: str):
    # Get all .json files without 'fulltext' prefix
    json_filepaths = [str(file) for file in Path(directory_path).rglob("*.json") if not file.name.startswith("fulltext")]
    
    # Create a set of filenames of fulltext files
    fulltext_files = {str(file).replace("fulltext_", "") for file in Path(directory_path).rglob("fulltext_*.json")}
    
    # Filter out files that have a corresponding fulltext version
    remaining_files = [file for file in json_filepaths if file not in fulltext_files]
    
    return remaining_files

In [ ]:
def to_json(filepath: str, export_file: list[dict], verbose: bool):
    # Modify filepath for export
    directory, filename = path.split(filepath)
    export_filename = f"fulltext_{filename}"
    export_filepath = path.join(directory, export_filename)

    # Export as .json
    with open(export_filepath, 'w') as json_file:
        json.dump(export_file, json_file, indent=4)

    if verbose:
        print(f"Processed '{export_filepath}'")

In [15]:
def fall_back_scrape_full_text(url: str):
    # Fall-back
    # Load HTML content
    response_fallback = requests.get(url)

    # Create a BeautifulSoup object
    soup_fallback = BeautifulSoup(response_fallback.text, 'html.parser')

    # Extract text from HTML
    article_container = soup_fallback.find('article')

    # Extract text from found element
    if article_container:
        full_text = "\n".join([p.get_text(strip=True) for p in article_container.find_all('p')])  # Extract paragraphs
    else:
        full_text = "Article content not found"

    return full_text

In [16]:
def scrape_nyt_url(json_filepaths: list[str]):
    for file in json_filepaths:
        # Open .json
        with open(file, "r") as f:
            response = json.load(f)  # Load JSON data into a dictionary

        # ---

        # Get all web urls
        web_url_list = [article['web_url'] for article in response['response']['docs']]

        # List to store article data for export
        list_of_dict = []

        # ---
        
        # Iterate through web urls
        for url in web_url_list:
            # Define cookies
            cookies = {
                'NYT-S': '0^CBoSMgjA89K9BhCXytW9BhoSMS3vHQ4I8sRMXKS37rBEiuLmIJyp_IIBKgIeVTjogcO9BkIAGkBmO6AJFtSiXUdFrL50D0JoH5sHkKVbO19xpSjKgTTkpxBPM9e7MHrS1G-EytgLWjjyD9ZkaBAJXgUfltOY8TEN',
                'nyt-a': 'jp4fDxi0FZZqELYzNbf4yQ'
            }

            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
            }

            # Use session with cookies
            session = requests.Session()
            response = session.get(url, headers=headers, cookies=cookies)

            # Parse the content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract title (either <h1> or <title>)
            title = soup.find('h1')
            if not title:
                title = soup.find('title')

            # Extract full article text (either from <section> or <article>)
            article_container = soup.find('section', {'name': 'articleBody'} or soup.find('article'))
            if article_container:
                full_text = "\n".join([p.get_text(strip=True) for p in article_container.find_all('p')])  # Extract paragraphs
            else:
                # Fall-back
                full_text = fall_back_scrape_full_text(url)

            # Prepare result as a dictionary
            result = {
                'url': url,
                'title': title.get_text(strip=True) if title else 'Title not found',
                'fulltext': full_text
            }

            list_of_dict.append(result)

            # Introduce a random delay between 1 and 3 seconds
            time.sleep(random.uniform(1, 3))

        to_json(filepath=file, export_file=list_of_dict, verbose=1)

### Execution

In [22]:
# Get filepaths
# json_filepaths = get_filepaths(directory_path = "../../data/raw/")
# print(len(json_filepaths))
# json_filepaths[0:3]

# Get remaining filepaths
json_filepaths = filter_scraped_files(directory_path = "../../data/raw/")
print(len(json_filepaths))
json_filepaths[0:3]


102


['../../data/raw/2021/organizations_Apple_Inc_mth08_pg1.json',
 '../../data/raw/2021/organizations_Apple_Inc_mth09_pg2.json',
 '../../data/raw/2021/organizations_Apple_Inc_mth07_pg0.json']

In [23]:
scrape_nyt_url(json_filepaths)

Processed 'fulltext_organizations_Apple_Inc_mth08_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth09_pg2.json'
Processed 'fulltext_organizations_Apple_Inc_mth07_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth06_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth10_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth10_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth11_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth07_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth06_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth09_pg3.json'
Processed 'fulltext_organizations_Apple_Inc_mth01_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth04_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth12_pg0.json'
Processed 'fulltext_organizations_Apple_Inc_mth12_pg1.json'
Processed 'fulltext_organizations_Apple_Inc_mth01_pg2.json'
Processed 'fulltext_organizations_Apple_Inc_mth04_pg0.json'
Processed 'fulltext_organizations_Apple_

In [ ]:
for file in json_files:
    # Open .json
    with open(file, "r") as f:
        response = json.load(f)  # Load JSON data into a dictionary

    # ---

    # Get all web urls for given .json
    web_url_list = []
    list_of_dict = [] # for export
    articles = response['response']['docs']

    for article in articles:
        web_url_list.append(article['web_url'])

    # ---

    # Extract full text using BeautifulSoup
    for url in web_url_list:
        response = requests.get(url)
        html_content = response.text

        # Create a BeautifulSoup object
        soup = bs(html_content, 'html.parser')

        # Extract text from HTML
        title_container = soup.find('title')
        article_container = soup.find('article')

        # Extract text from found element
        if article_container:
            text_content = "\n".join([p.get_text(strip=True) for p in article_container.find_all('p')])  # Extract paragraphs
        else:
            text_content = "No article content found."

        title_content = title_container.get_text(strip=True)

        # Text Export
        export_dict = {
            'headline': title_content,
            'web_url': url,
            'full_text': text_content
        }

        list_of_dict.append(export_dict)

    # Modify filepath for export
    directory, filename = path.split(file)
    export_filename = f"fulltext_{filename}"
    export_filepath = path.join(directory, export_filename)

    # Export as .json
    with open(export_filepath, 'w') as json_file:
        json.dump(list_of_dict, json_file, indent=4)
    
    print(f"Processed '{file}'")

Processed '../../data/raw/2022/organizations_Apple_Inc_mth04_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth05_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth12_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth12_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth04_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth05_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth09_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth08_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth03_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth02_pg0.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth03_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth02_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth09_pg1.json'
Processed '../../data/raw/2022/organizations_Apple_Inc_mth07_pg0.json'
Proces

KeyboardInterrupt: 

In [ ]:
def extract_article_full_text(json_files):
    for file in json_files:
        # Open .json and load data
        with open(file, "r") as f:
            response = json.load(f)

        # Get all web urls
        web_url_list = [article['web_url'] for article in response['response']['docs']]

        # List to store article data for export
        list_of_dict = []

        # Process each URL
        for url in web_url_list:
            response = requests.get(url)
            html_content = response.text

            # Create BeautifulSoup object
            soup = bs(html_content, 'html.parser')

            # Extract title and full article text
            title_container = soup.find('title')
            article_container = soup.find('article')

            if article_container:
                full_text = "\n".join([p.get_text(strip=True) for p in article_container.find_all('p')])
            else:
                full_text = "No article content found."

            title_content = title_container.get_text(strip=True) if title_container else "No title found"

            # Prepare article data
            export_dict = {
                'headline': title_content,
                'web_url': url,
                'full_text': full_text
            }

            list_of_dict.append(export_dict)

        # Prepare filepath for export
        directory, filename = path.split(file)
        export_filename = f"fulltext_{filename}"
        export_filepath = path.join(directory, export_filename)

        # Export data to .json
        with open(export_filepath, 'w') as json_file:
            json.dump(list_of_dict, json_file, indent=4)

        print(f"Processed '{file}' and exported to '{export_filename}'")

# Example usage
json_files = ['file1.json', 'file2.json']  # Replace with your actual file paths
extract_article_full_text(json_files)
